# Settings

## Setup environment

### (Optional) If you are on google colab,

- Visit google drive (https://drive.google.com)
- Create a folder (example: hyu)
- Upload this entire folder (i.e., HYU-ECL3003) to the folder
- Open `train_detector.ipynb` on google drive (this action will open google colab session)
- Switch runtime to `T4 GPU`


### Common settings

```bash
$ python3 -m venv venv/ecl3003
$ source venv/ecl3003/bin/activate
$ pip install -r rover/requirements.txt
```

### Download your dataset from CVAT 
```bash
$ mkdir -p ../../datasets
$ unzip -d  ../../datasets/ ECL3003-dataset.zip 
```

### Set validation set (or you can set train.txt for testing)
```bash
$ echo "val: train.txt" >> ../../datasets/data.yaml
```

In [ ]:
# ### (Optional) If you are on google colab,

# # Mount google drive
# from google.colab import drive
# drive.mount('/content/drive/')

# # Move to the new directory
# cd /content/drive/MyDrive/hyu

# Extract images

In [ ]:
import cv2
import os

capture = cv2.VideoCapture("test_video.avi")
assert capture.isOpened(), "Cannot open the video file."

num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Prepare folders
img_filename_fmt = 'data/images/train/frame_{:06d}.png'
dirname = os.path.dirname(img_filename_fmt)
os.makedirs(dirname, exist_ok=True)

for ii in range(num_frames):
    _, frame = capture.read()
    cv2.imwrite(img_filename_fmt.format(ii), frame)

capture.release()

# Make a symbolic link

In [ ]:
import os
os.symlink('../../../datasets/labels', 'data/labels')

# Train a YOLO model

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 dataset for 100 epochs
train_results = model.train(
    data="../../datasets/data.yaml",  # Path to dataset configuration file
    epochs=200,  # Number of training epochs
    imgsz=640,  # Image size for training
    device=0,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    optimizer='Adam',
    lr0=1e-2,
)

# Make an inference

In [ ]:
# Evaluate the model's performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model("data/images/train/frame_000000.png")  # Predict on an image
results[0].show()  # Display results

In [ ]:

# # Export the model to ONNX format for deployment
# path = model.export(format="onnx")  # Returns the path to the exported model